In [37]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [38]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [39]:
# Add name of goolge spread sheet here
sheet_name = '0708 Wilder Check-in'

# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open(sheet_name).sheet1

In [40]:
# Put the full file name for the smartwaiver data here
smartwaiver = 'smartwaiver-176635-64a89d82ee6cc.csv'

# Load the data from your CSV file
data = pd.read_csv(smartwaiver)

In [41]:
# Convert 'Waiver Date' column to datetime
data['Date Completed (UTC)'] = pd.to_datetime(data['Date Completed (UTC)'])

In [42]:
# Get the current date
now = datetime.now()

# Calculate the date one year ago
one_year_ago = now - timedelta(days=365)

In [43]:
# Filter the data to get volunteers who signed a waiver in the past year
filtered_data = data[data['Date Completed (UTC)'] >= one_year_ago]

In [44]:
# View filtered dataframe
filtered_data.head(5)

,First,Middle,Last,Date of Birth,Date Completed (UTC),Waiver ID,Status,Check-ins,Gender,Phone,...,Parent/Guardian First Name,Parent/Guardian Middle Name,Parent/Guardian Last Name,Parent/Guardian Phone,Parent/Guardian Relationship,Email,signing_type,template_type,Title of Document,typed_font
0,Sean,NaN,Kim,06/30/2007,2023-06-19 20:35:17,9SCi3sJWhZbHerxrUgQ2T5,Completed Online,0,NaN,NaN,...,Yijin,NaN,Yun,NaN,NaN,yijin.yun@gmail.com,draw,1,SCMTS + CA State Parks,dancing
1,Brian,NaN,Hackett,NaN,2023-06-20 12:12:30,7NQ2r6SEW2qhXZ36pNPfxC,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,bhackett1024@gmail.com,draw,1,SCMTS + CA State Parks,dancing
2,Tara,NaN,Thomas,NaN,2023-06-20 18:39:03,48jNvvkRzWWkij2iBvzuWD,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,tanthoma@ucsc.edu,draw,1,SCMTS + CA State Parks,dancing
3,Philip,NaN,Fifer,NaN,2023-06-20 23:17:51,ZdQujqF5KtYgBaX6ki4rTk,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,pfifer@canyon.com,draw,1,SCMTS + CA State Parks,dancing
4,Matthew,NaN,Noble,NaN,2023-06-21 01:01:35,rdSjd9wpAEn2qWE5pvAY6F,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,mnoble@canyon.com,draw,1,SCMTS + CA State Parks,dancing


In [45]:
# Get all records from the Google Sheet
records = sheet.get_all_records()

# Create an empty DataFrame to store unmatched names
unmatched_names = pd.DataFrame(columns=['First Name', 'Last Name', 'Preferred Email'])

In [46]:

# Get the list of column names
headers = sheet.row_values(1)

# Find the index of the 'SCMTS Waiver' column (Python index starts at 0 so we need to add 1 to match gspread formatting)
waiver_column_index = headers.index('Waiver') + 1

# Iterate over the filtered data
for index, row in filtered_data.iterrows():
    
    # Find the volunteer in the Google Spreadsheet
    matched_records = [record for record in records if record['Contact Name'].split(", ")[1] == row['First'] and record['Contact Name'].split(", ")[0] == row['Last']]

    if matched_records:
          
        # If there is a match check the box
        record = matched_records[0]
        
        row_number = records.index(record) + 2 # Adds 2 because gspread row index starts at 1 and we have a header row

        # Check the box in the 4th column
        sheet.update_cell(row_number, 4, True) # The number 3 reflects the column number

In [47]:
# Iterate over the records from the Google Sheets
for record in records:
    # Check if the box is still 'False'
    if record['Waiver'] == 'FALSE':  # gspread recognizes empty checkboxes as a string 'FALSE'
        # If it's still 'False', add the contact name and email to the DataFrame
        name_parts = record['Contact Name'].split(", ")
        if len(name_parts) == 2: # Making sure the name is in "Last, First" format
            unmatched_names = unmatched_names.append({'First': name_parts[1], 'Last': name_parts[0], 'Email': record['Preferred Email']}, ignore_index=True)

In [48]:
# View dataframe for individuals that did not sign the waiver (to be used for smart_gmail)
unmatched_names

,First Name,Last Name,Preferred Email,Email,First,Last
0,NaN,NaN,NaN,baris98aslan@protonmail.com,Volkan,Aslan
1,NaN,NaN,NaN,melissa_egge@yahoo.com,Patrick,Egge
2,NaN,NaN,NaN,brettmhatch@gmail.com,Brett,Hatch
3,NaN,NaN,NaN,esanders@santacruzcoe.org,Sarah,Sanders
4,NaN,NaN,NaN,tiller.nathan@gmail.com,Nathan,Tiller
